In [14]:
from aip import AipOcr
import cv2
import math
import re
import os
import numpy as np
from scipy import misc, ndimage
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import argparse

APP_ID = '15589371'
API_KEY = '2ch5BU9uXXGAfrCsgINAqc4Q'
SECRET_KEY = 'tKnG9larHOgaZ6eEqMbAuMd54tiLbcHA'

FILE_PATH = './data/Business-Receipt/'
#ORIGINAL_IMAGE = './data/Business-Receipt/20190224084417475.jpg'
ORIGINAL_IMAGE = '20190304012952250.jpg'
PROCESSED_IMAGE = 'processed.jpg'

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

def onadaptivethresh(filePath, outputpath):
    
    img = cv2.imread(filePath)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gauss = cv2.GaussianBlur(gray, (3,3), 1)
    maxvalue = 255

    value = cv2.getTrackbarPos('value', 'Threshold')
    if(value < 3):
        value = 3
    if(value % 2 == 0):
        value += 1
    args = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, value, 1)
    gaus = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, value, 1)
    cv2.imwrite(outputpath, gaus)
    #return gaus

def rotate(original_file, output_file):
    
    img = cv2.imread(original_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
 
    # 霍夫变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 0)
    rotate_angle = 0
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if x1 == x2 or y1 == y2:
            continue
        t = float(y2 - y1) / (x2 - x1)
        rotate_angle = math.degrees(math.atan(t))
        if rotate_angle > 45:
            rotate_angle = -90 + rotate_angle
        elif rotate_angle < -45:
            rotate_angle = 90 + rotate_angle
    #print("rotate_angle : "+str(rotate_angle))
    rotate_img = ndimage.rotate(img, rotate_angle)
    #cv2.imencode('.jpg',rotate_img)[1].tofile(output_file)
    cv2.imwrite(output_file,rotate_img)


def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()
    
def extract_key_information(all_information):
    key_information = {}
    
    key1 = '回单编号'
    key2 = '付款人户名'
    key3 = '付款人账号'
    key4 = '付款人开户行'
    key5 = '收款人户名'
    key6 = '收款人账号'
    key7 = '收款人开户行'
    key8 = '金额'
    key9 = '日期'
    
    
    #提取“回单编号”
    for item in all_information:
        if key1 in item:
            key_information[key1] = re.search('([0-9]+)', item).group()
            break
#     if key1 not in key_information:
#         key_information[key1] = ''
    #提取付款人户名
    
    for item in all_information:
        if key2 in item:
            key_information[key2] = item[item.find(key2)+len(key2)+1:]
            break
    if key2 not in key_information:
        key_information[key2] = ''
    #提取付款人账号
    
    for item in all_information:
        if key3 in item:
            key_information[key3] = re.search('([0-9]+)', item).group()
            break
    if key3 not in key_information:
        key_information[key3] = ''
    
    #提取付款人开户行    
    for item in all_information:
        if key4 in item:
            key_information[key4] = item[item.find(key4)+len(key4)+1:]
    ##处理开户行内容超过一行的情况
    for i in range(len(all_information)):
        if key5 in all_information[i]:

            if key3 not in all_information[i-1] and key4 not in all_information[i-1]:
                key_information[key4] += all_information[i-1]
            else:
                break
    if key4 not in key_information:
        key_information[key4] = ''
        
    #提取收款人户名    
    for item in all_information:
        if key5 in item:
            key_information[key5] = item[item.find(key5)+len(key5)+1:]
            break
    if key5 not in key_information:
        key_information[key5] = ''
        
    #提取收款人账号    
    for item in all_information:
        if key6 in item:
            key_information[key6] = re.search('([0-9]+)', item).group() if re.search('([0-9]+)', item) != None else ''
            break

            
    #提取收款人开户行
    for item in all_information:
        if key7 in item:
            key_information[key7] = item[item.find(key7)+len(key7)+1:]
            break
    ##处理开户行内容超过一行的情况
    for i in range(len(all_information)):
        if key6 in all_information[i]:
            #print(all_information[i+1])

            if key8 not in all_information[i+1]:
                key_information[key7] += all_information[i+1]
            else:
                break
    if key7 not in key_information:
        key_information[key7] = ''
        
    #提取金额    
    for item in all_information:
        if '小写' in item or re.search('[(0-9)+][元]', item) != None:
            key_information[key8] = item[item.find(key8)+len(key8)+2:]
            break
    if key8 not in key_information:
        key_information[key8] = ''
        
        
    #提取日期
    for item in all_information:
        if key9 in item and '打印日期' not in item:
            key_information[key9] = item[item.find(key9)+len(key9)+1:]
            break
    if key9 not in key_information:
        key_information[key9] = ''
    
    return key_information
    
def preprocess(image_file, output_filename):
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    docCnt = None
    
    if len(cnts) > 0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02*peri, True)
            if len(approx) == 4:
                docCnt = approx
                break
                
    paper = four_point_transform(img, docCnt.reshape(4, 2))
    warped = four_point_transform(gray, docCnt.reshape(4, 2))
    cv2.imwrite(output_filename, warped)
    
def creat_database(database_name):
    conn = sqlite3.connect('bill-information.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE 业务回单
                 (id int primary key NOT NULL,
                 回单编号 text NOT NULL,
                 付款人户名 text,
                 付款人账号 text,
                 付款人开户行 test,
                 收款人户名 text,
                 收款人账号 text,
                 收款人开户行 text,
                 金额 text NOT NULL,
                 日期 text NOT NULL)''')

    conn.commit()
    conn.close()
    

#all_information = []
#key_information = []
#file_list = os.listdir(FILE_PATH)
#for file in file_list:
#    file_name = FILE_PATH + file
#    file_format = file_name.split('.')[-1].lower() #提取文件格式，用于判断该文件是否是图片
#    if file_format == 'jpg' or file_format == 'png' or file_format == 'jpeg' or file_format == 'bmp':    #若文件是图片格式，才继续后续处理步骤
#        preprocess(file, PROCESSED_IMAGE)          #预处理
#        image = get_file_content(PROCESSED_IMAGE)  #读取图像
#       result = client.accurate(image)            #API对图像进行识别
#        for i in range(len(result['words_result'])):
#            all_information.append(result['words_result'][i]['words'])  #获取识别得到的每一行的结果
            
#        key_information_dict = extract_key_information(all_information)  #根据要求提取关键信息
#        key_information_dict['文件路径'] = file_name
#        key_information.append(key_information_dict)  
            

            
            
#图像调正
#rotate(ORIGINAL_IMAGE, PROCESSED_IMAGE)

#preprocess(ORIGINAL_IMAGE,PROCESSED_IMAGE)

#图像二值化
#onadaptivethresh(PROCESSED_IMAGE, PROCESSED_IMAGE)

image = get_file_content(PROCESSED_IMAGE)


para_options = {}
para_options['language_type'] = 'CHN_ENG'
para_options['detect_direction'] = 'true'
para_options['detect_language'] = 'true'
para_options['probability'] = 'true'

result = client.accurate(image)
all_information = []
 
for i in range(len(result['words_result'])):
    #num_line = str(i+1) + '  '
    all_information.append(result['words_result'][i]['words'])
    #print(str(i)+' ' +all_information[i])
key_information = extract_key_information(all_information)

for item in key_information.items():
    print(item)

('付款人户名', '延长油田股份有限公司靖边采油厂')
('日期', '2017年10月17日')
('回单编号', '17290000002')
('收款人户名', '涿州铁新泵业科技发展有限公司')
('付款人账号', '01')
('金额', '15,000.00元')
('付款人开户行', '工行陕西省分行西安高新技术开发区支行营业室')
('收款人开户行', '保定涿州支行营业室')
('收款人账号', '0409020009245098508')


In [49]:
import re
def extract_key_information(all_information):
    key_information = {}
    
    #提取“回单编号”
    key1 = '回单编号'
    key2 = '付款人户名'
    key3 = '付款人账号'
    key4 = '付款人开户行'
    key5 = '收款人户名'
    key6 = '收款人账号'
    key7 = '收款人开户行'
    key8 = '金额'
    key9 = '日期'
    
    for item in all_information:
        if key1 in item:
            key_information[key1] = re.search('([0-9]+)', item).group()
            break
    #提取付款人户名
    
    for item in all_information:
        if key2 in item:
            key_information[key2] = item[item.find(key2)+len(key2)+1:]
            break
    #提取付款人账号
    
    for item in all_information:
        if key3 in item:
            key_information[key3] = re.search('([0-9]+)', item).group()
            break
    #提取付款人开户行
    
    for item in all_information:
        if key4 in item:
            key_information[key4] = item[item.find(key4)+len(key4)+1:]
                        
    
    for i in range(len(all_information)):
        if key5 in all_information[i]:
            if key3 not in all_information[i-1]:
                key_information[key4] += all_information[i-1]
    
            else:
                break
          
    #提取收款人户名
    
    for item in all_information:
        if key5 in item:
            key_information[key5] = item[item.find(key5)+len(key5)+1:]
            break
    #提取收款人账号
    
    for item in all_information:
        if key6 in item:
            key_information[key6] = re.search('([0-9]+)', item).group()
            break
    #提取收款人开户行
    
    for item in all_information:
        if key7 in item:
            key_information[key7] = item[item.find(key7)+len(key7)+1:]
            break
    #提取金额
    
    for item in all_information:
        if '小写' in item:
            key_information[key8] = item[item.find(key8)+len(key8)+1:]
            break
    #提取日期
    
    for item in all_information:
        if key9 in item and '打印日期' not in item:
            key_information[key9] = item[item.find(key9)+len(key9)+1:]
            break
    return key_information
for item in key_information.items():
    print(item)
    

('回单编号', '17314000001')
('收款人户名', '涿州铁新泵业科技发展有限公司')
('付款人开户行', '中国农业银行延安分行库管清算中心')
('付款人户名', '安塞宏达工贸有限责任公司')
('日期', '2017年11月10日')
('收款人开户行', '保定涿州支行营业室')
('收款人账号', '0409020009245098508')
('付款人账号', '26930101040004849')
('金额', ':50000.00元')


In [35]:
def rotate(original_file, output_file):
    
    img = cv2.imread(original_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
 
    # 霍夫变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 0)
    print(lines)
    rotate_angle = 0
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if x1 == x2 or y1 == y2:
            continue
        t = float(y2 - y1) / (x2 - x1)
        rotate_angle = math.degrees(math.atan(t))
        if rotate_angle > 45:
            rotate_angle = -90 + rotate_angle
        elif rotate_angle < -45:
            rotate_angle = 90 + rotate_angle
    #print("rotate_angle : "+str(rotate_angle))
    rotate_img = ndimage.rotate(img, rotate_angle)
    cv2.imwrite(output_file,rotate_img)
    
ORIGINAL_IMAGE = '20190224084441867.jpg'
PROCESSED_IMAGE = 'temp.jpg'

rotate(ORIGINAL_IMAGE, PROCESSED_IMAGE)

[[[ 2.2670000e+03  1.5882496e+00]]

 [[ 1.9270000e+03  1.5882496e+00]]

 [[ 1.6350000e+03  1.5882496e+00]]

 ...

 [[-1.7820000e+03  2.6005406e+00]]

 [[-1.9740000e+03  2.6179938e+00]]

 [[-2.0330000e+03  2.6354473e+00]]]


输入图像自适应二值化处理

In [ ]:
import cv2

IMAGE_FILE = '20190219161556.jpg'
img = cv2.imread(IMAGE_FILE)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gauss = cv2.GaussianBlur(gray, (3,3), 1)
maxvalue = 255


value = cv2.getTrackbarPos('value', 'Threshold')
if(value < 3):
    value = 3
if(value % 2 == 0):
    value += 1
args = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, value, 1)
gaus = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, value, 1)
cv2.imshow('Args', args)
cv2.imshow('Gaus', gaus)
cv2.imshow('IMG', img)
cv2.waitKey(0)

In [17]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import argparse
import cv2

#IMAGE_FILE = '20190224084441867.jpg'
IMAGE_FILE = './data/Business-Receipt/20190224084457195.jpg'
image = cv2.imread(IMAGE_FILE)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5,5), 0)
edged = cv2.Canny(blurred, 75, 200)

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
docCnt = None

if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        if len(approx) == 4:
            docCnt = approx
            break
            
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))
#cv2.imshow('original', image)
cv2.imwrite('Exam.jpg', paper)
cv2.imwrite('Exam1.jpg', warped)
#cv2.waitKey(0)

True

In [5]:
import os
import cv2
import math
import random
import numpy as np
from scipy import misc, ndimage
import matplotlib.pyplot as plt
IMAGE_FILE = '\两类档案和需要提取的数据项说明\两类档案和需要提取的数据项说明\ICBC业务回单\20190224084335732.jpg'
 
def rotate(original_file, output_file):
    
    img = cv2.imread(original_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
 
    # 霍夫变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 0)
    
    rotate_angle = 0
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if x1 == x2 or y1 == y2:
            continue
        t = float(y2 - y1) / (x2 - x1)
        rotate_angle = math.degrees(math.atan(t))
        if rotate_angle > 45:
            rotate_angle = -90 + rotate_angle
        elif rotate_angle < -45:
            rotate_angle = 90 + rotate_angle
    print("rotate_angle : " + str(rotate_angle))
    rotate_img = ndimage.rotate(img, rotate_angle)
    cv2.imwrite(output_file,rotate_img)
    cv2.imshow("img", rotate_img)
    cv2.waitKey(0)

关键信息存入数据库

In [5]:
import sqlite3

conn = sqlite3.connect('bill-information.db')
c = conn.cursor()
c.execute('''CREATE TABLE 业务回单
             (回单编号 text primary key NOT NULL,
             付款人户名 text,
             付款人账号 text,
             付款人开户行 test,
             收款人户名 text,
             收款人账号 text,
             收款人开户行 text,
             金额 text,
             日期 text,
             文件路径 text)''')

conn.commit()
conn.close()

In [17]:
import sqlite3

bill_information = (key_information['回单编号'],
                    key_information['付款人户名'],
                    key_information['付款人账号'],
                    key_information['付款人开户行'],
                    key_information['收款人户名'],
                    key_information['收款人账号'],
                    key_information['收款人开户行'],
                    key_information['金额'],
                    key_information['日期'],
                    key_information['文件路径'])
conn = sqlite3.connect('bill-information.db')
c = conn.cursor()
c.execute('''INSERT INTO 业务回单 (回单编号, 
                                   付款人户名,
                                   付款人账号,
                                   付款人开户行,
                                   收款人户名,
                                   收款人账号,
                                   收款人开户行,
                                   金额,
                                   日期,
                                   文件路径)
             VALUES (?, ?, ?, ?, ?,?,?,?,?,?,?)''', bill_information)
#c.execute('INSERT INTO 业务回单 VALUES (?, ?, ?, ?, ?,?,?,?,?)', bill_information)
conn.commit()
conn.close()

In [5]:
from aip import AipOcr
import cv2
import math
import re
import os
import numpy as np
from scipy import misc, ndimage
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import argparse
import sqlite3

APP_ID = '15589371'
API_KEY = '2ch5BU9uXXGAfrCsgINAqc4Q'
SECRET_KEY = 'tKnG9larHOgaZ6eEqMbAuMd54tiLbcHA'

FILE_PATH = './data/Business-Receipt'
ORIGINAL_IMAGE = './data/Business-Receipt/20190224084417475.jpg'
PROCESSED_IMAGE = './temp/processed.jpg'

client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

def onadaptivethresh(filePath, outputpath):
    
    img = cv2.imread(filePath)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gauss = cv2.GaussianBlur(gray, (3,3), 1)
    maxvalue = 255

    value = cv2.getTrackbarPos('value', 'Threshold')
    if(value < 3):
        value = 3
    if(value % 2 == 0):
        value += 1
    args = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, value, 1)
    gaus = cv2.adaptiveThreshold(gauss, maxvalue, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, value, 1)
    cv2.imwrite(outputpath, gaus)
    #return gaus

def rotate(original_file, output_file):
    
    img = cv2.imread(original_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
 
    # 霍夫变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 0)
    rotate_angle = 0
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        if x1 == x2 or y1 == y2:
            continue
        t = float(y2 - y1) / (x2 - x1)
        rotate_angle = math.degrees(math.atan(t))
        if rotate_angle > 45:
            rotate_angle = -90 + rotate_angle
        elif rotate_angle < -45:
            rotate_angle = 90 + rotate_angle
    #print("rotate_angle : "+str(rotate_angle))
    rotate_img = ndimage.rotate(img, rotate_angle)
    #cv2.imencode('.jpg',rotate_img)[1].tofile(output_file)
    cv2.imwrite(output_file,rotate_img)


def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()
    
def extract_key_information(all_information):
    key_information = {}
    
    key1 = '回单编号'
    key2 = '付款人户名'
    key3 = '付款人账号'
    key4 = '付款人开户行'
    key5 = '收款人户名'
    key6 = '收款人账号'
    key7 = '收款人开户行'
    key8 = '金额'
    key9 = '日期'
    
    
    #提取“回单编号”
    for item in all_information:
        if key1 in item:
            key_information[key1] = re.search('([0-9]+)', item).group()
            break

    #提取付款人户名
    
    for item in all_information:
        if key2 in item:
            key_information[key2] = item[item.find(key2)+len(key2)+1:]
            break
    if key2 not in key_information:
        key_information[key2] = ''
    #提取付款人账号
    
    for item in all_information:
        if key3 in item:
            key_information[key3] = re.search('([0-9]+)', item).group()
            break
    if key3 not in key_information:
        key_information[key3] = ''
    
    #提取付款人开户行    
    for item in all_information:
        if key4 in item:
            key_information[key4] = item[item.find(key4)+len(key4)+1:]
    ##处理开户行内容超过一行的情况
    for i in range(len(all_information)):
        if key5 in all_information[i]:

            if key3 not in all_information[i-1] and key4 not in all_information[i-1]:
                key_information[key4] += all_information[i-1]
            else:
                break
    if key4 not in key_information:
        key_information[key4] = ''
        
    #提取收款人户名    
    for item in all_information:
        if key5 in item:
            key_information[key5] = item[item.find(key5)+len(key5)+1:]
            break
    if key5 not in key_information:
        key_information[key5] = ''
        
    #提取收款人账号    
    for item in all_information:
        if key6 in item:
            key_information[key6] = re.search('([0-9]+)', item).group() if re.search('([0-9]+)', item) != None else ''
            break
    if key6 not in key_information:
        key_information[key6] = ''
            
    #提取收款人开户行
    for item in all_information:
        if key7 in item:
            key_information[key7] = item[item.find(key7)+len(key7)+1:]
            break
    ##处理开户行内容超过一行的情况
    for i in range(len(all_information)):
        if key6 in all_information[i]:
            #print(all_information[i+1])

            if key8 not in all_information[i+1]:
                key_information[key7] += all_information[i+1]
            else:
                break
    if key7 not in key_information:
        key_information[key7] = ''
        
    #提取金额    
    for item in all_information:
        if '小写' in item or re.search('[(0-9)+][元]', item) != None:
            key_information[key8] = item[item.find(key8)+len(key8)+2:]
            break
    if key8 not in key_information:
        key_information[key8] = ''
        
        
    #提取日期
    for item in all_information:
        if key9 in item and '打印日期' not in item:
            key_information[key9] = item[item.find(key9)+len(key9)+1:]
            break
    if key9 not in key_information:
        key_information[key9] = ''
    
    return key_information

def sift_kp(image):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #sift = cv2.xfeatures2d_SIFT.create()
    sift = cv2.xfeatures2d.SIFT_create()
    kp,des = sift.detectAndCompute(image,None)
    kp_image = cv2.drawKeypoints(gray_image,kp,None)
    return kp_image,kp,des

def get_good_match(des1,des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    return good

def siftImageAlignment(img1,img2):
   _,kp1,des1 = sift_kp(img1)
   _,kp2,des2 = sift_kp(img2)
   goodMatch = get_good_match(des1,des2)
   if len(goodMatch) > 4:
       ptsA= np.float32([kp1[m.queryIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
       ptsB = np.float32([kp2[m.trainIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
       ransacReprojThreshold = 4
       H, status =cv2.findHomography(ptsA,ptsB,cv2.RANSAC,ransacReprojThreshold);
       imgOut = cv2.warpPerspective(img2, H, (img1.shape[1],img1.shape[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
   return imgOut,H,status
    
def preprocess(image_file, output_filename):
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    docCnt = None
    
    if len(cnts) > 0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02*peri, True)
            if len(approx) == 4:
                docCnt = approx
                break
                
    paper = four_point_transform(img, docCnt.reshape(4, 2))
    warped = four_point_transform(gray, docCnt.reshape(4, 2))
    cv2.imwrite(output_filename, warped)
    
    
def creat_database(database_name):
    conn = sqlite3.connect(database_name)
    c = conn.cursor()
    c.execute('''CREATE TABLE 业务回单
                 (
                 回单编号 text primary key NOT NULL,
                 付款人户名 text,
                 付款人账号 text,
                 付款人开户行 test,
                 收款人户名 text,
                 收款人账号 text,
                 收款人开户行 text,
                 金额 text,
                 日期 text,
                 文件路径 text)''')

    conn.commit()
    conn.close()
    
#数据存入数据库
def insert_into_database(key_information): 
    bill_information = (key_information['回单编号'],
                        key_information['付款人户名'],
                        key_information['付款人账号'],
                        key_information['付款人开户行'],
                        key_information['收款人户名'],
                        key_information['收款人账号'],
                        key_information['收款人开户行'],
                        key_information['金额'],
                        key_information['日期'],
                        key_information['文件路径'])
    conn = sqlite3.connect('bill-information.db')
    c = conn.cursor()
    
    c.execute('''INSERT INTO 业务回单 (回单编号, 
                                       付款人户名,
                                       付款人账号,
                                       付款人开户行,
                                       收款人户名,
                                       收款人账号,
                                       收款人开户行,
                                       金额,
                                       日期,
                                       文件路径)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', bill_information)
    #c.execute('INSERT INTO 业务回单 VALUES (?, ?, ?, ?, ?,?,?,?,?)', bill_information)
    conn.commit()
    conn.close()
    

key_information = []
file_list = os.listdir(FILE_PATH)
for file in file_list:
    all_information = []
    file_name = FILE_PATH + '/' + file
    file_format = file_name.split('.')[-1].lower() #提取文件格式，用于判断该文件是否是图片
    if file_format == 'jpg' or file_format == 'png' or file_format == 'jpeg' or file_format == 'bmp':    #若文件是图片格式，才继续后续处理步骤
        #图像调正
        preprcessed_image_name = "./temp/test_" + file
        preprocess(file_name, preprcessed_image_name)          #预处理
        #rotate(ORIGINAL_IMAGE, PROCESSED_IMAGE)
        #图像二值化
        #onadaptivethresh(PROCESSED_IMAGE, PROCESSED_IMAGE)
        image = get_file_content(preprcessed_image_name)  #读取图像
        result = {}
        result = client.accurate(image)            #API对图像进行识别
        if 'error_code' in result:
            result = client.basicAccurate(image)
        #print(result)
        if 'error_code' not in result:
            #print(len(result['words_result']))
            for i in range(len(result['words_result'])):
                #print((result['words_result'][i]['words']))
                all_information.append(result['words_result'][i]['words'])  #获取识别得到的每一行的结果
        else:
            error_msg = "API调用失败:"+str(result['error_msg'])
            print(error_msg)
        #print('------')
        #print(all_information)
        key_information_dict = extract_key_information(all_information)  #根据要求提取关键信息
        key_information_dict['文件路径'] = file_name
        key_information.append(key_information_dict)

#创建数据库列表
DATA_BASE_NAME = 'bill-information.db'
creat_database(DATA_BASE_NAME)

for i in range(len(key_information)):
    try: 
        insert_into_database(key_information[i])
    except KeyError:
        print(key_information[i]['文件路径'] + '识别失败！')

    
    
    #if '回单编号' in key_information[i]:
        #insert_into_database(key_information[i])
    #else:
        #print(key_information[i]['文件路径'] + '识别失败！')


0
1
2
3
4
5
6
7
./data/Business-Receipt/20190224084441867.jpg识别失败！
8
9
10


In [11]:

conn = sqlite3.connect('bill-information.db')
cur = conn.cursor()
c = cur.execute('SELECT * FROM 业务回单')
for row in c:
    print(row)
conn.close()

('17314000001', '安塞宏达工贸有限责任公司', '26930101040004849', '中国农业银行延安分行库管清算中心', '涿州铁新泵业科技发展有限公司', '0409020009245098508', '保定涿州支行营业室', '50,000.00元', '2017年11月10日', './data/Business-Receipt/20190224084335732.jpg')
('173100000', '涿州铁新泵业科技发展有限公司', '0409020009245098508', '保定涿州支行营业室', '', '', '', '10.00元', '2017年11月06日', './data/Business-Receipt/20190224084346196.jpg')
('17285000001', '涿州铁新泵业科技发展有限公司', '0409020009245098508', '保定涿州支行营业室', '涿州市桃园钢化玻璃厂', '100148062313', '中国银行涿州支行', '200,000.00元', '2017年10月12日', './data/Business-Receipt/20190224084354267.jpg')
('17284000002', '安塞宏达工贸有限责任公司', '26930101040004849', '中国农业银行延安分行库管清算中心', '涿州铁新泵业科技发展有限公司', '0409020009245098508', '保定涿州支行营业室', '20,000.00元', '2017年10月11日', './data/Business-Receipt/20190224084401475.jpg')
('17284000001', '北京金顺泰达机械加工有限责任公司', '0901000103000054058', '北京农村商业银行股份有限公司', '涿州铁新泵业科技发展有限公司', '0409020009245098508', '保定涿州支行营业室', '300,000.00元', '2017年10月11日', './data/Business-Receipt/20190224084409442.jpg')
('17286000002', '中国石化国际事业有限公司北京招标中心

In [19]:
import cv2
import numpy as np
import os
import imutils
from scipy import misc, ndimage
from imutils.perspective import four_point_transform
from imutils import contours

def sift_kp(image):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #sift = cv2.xfeatures2d_SIFT.create()
    sift = cv2.xfeatures2d.SIFT_create()
    kp,des = sift.detectAndCompute(image,None)
    kp_image = cv2.drawKeypoints(gray_image,kp,None)
    return kp_image,kp,des

def get_good_match(des1,des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    return good

def siftImageAlignment(img1,img2):
    img1 = cv2.imread(img1)    
    img2 = cv2.imread(img2)
    _,kp1,des1 = sift_kp(img1)
    _,kp2,des2 = sift_kp(img2)
    goodMatch = get_good_match(des1,des2)
    if len(goodMatch) > 4:
        ptsA= np.float32([kp1[m.queryIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
        ptsB = np.float32([kp2[m.trainIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
        ransacReprojThreshold = 4
        H, status =cv2.findHomography(ptsA,ptsB,cv2.RANSAC,ransacReprojThreshold);
        imgOut = cv2.warpPerspective(img2, H, (img1.shape[1],img1.shape[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        
    return imgOut
#     return imgOut,H,status


def preprocess(sift_image, output_filename):
#     img = cv2.imread(image_file)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(sift_image, (5,5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    docCnt = None
    
    if len(cnts) > 0:
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02*peri, True)
            if len(approx) == 4:
                docCnt = approx
                break
                
    #paper = four_point_transform(img, docCnt.reshape(4, 2))
    warped = four_point_transform(sift_image, docCnt.reshape(4, 2))
    cv2.imwrite(output_filename, warped)
    
def creat_database(database_name):
    conn = sqlite3.connect(database_name)
    c = conn.cursor()
    c.execute('''CREATE TABLE 业务回单
                 (
                 回单编号 text primary key NOT NULL,
                 付款人户名 text,
                 付款人账号 text,
                 付款人开户行 test,
                 收款人户名 text,
                 收款人账号 text,
                 收款人开户行 text,
                 金额 text,
                 日期 text,
                 文件路径 text)''')

    conn.commit()
    conn.close()
    
#数据存入数据库
def insert_into_database(key_information): 
    bill_information = (key_information['回单编号'],
                        key_information['付款人户名'],
                        key_information['付款人账号'],
                        key_information['付款人开户行'],
                        key_information['收款人户名'],
                        key_information['收款人账号'],
                        key_information['收款人开户行'],
                        key_information['金额'],
                        key_information['日期'],
                        key_information['文件路径'])
    conn = sqlite3.connect('bill-information.db')
    c = conn.cursor()
    
    c.execute('''INSERT INTO 业务回单 (回单编号, 
                                       付款人户名,
                                       付款人账号,
                                       付款人开户行,
                                       收款人户名,
                                       收款人账号,
                                       收款人开户行,
                                       金额,
                                       日期,
                                       文件路径)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', bill_information)
    #c.execute('INSERT INTO 业务回单 VALUES (?, ?, ?, ?, ?,?,?,?,?)', bill_information)
    conn.commit()
    conn.close()

# img1 = cv2.imread('./data/Business-Receipt/20190224084335732.jpg')
# img2 = cv2.imread('./data/Business-Receipt/20190224084441867.jpg')
FILE_PATH = './data/Business-Receipt/'
TEMPLATE = './data/Business-Receipt/template.jpg'

# result,_,_ = siftImageAlignment(img1,img2)
#allImg = np.concatenate((img1,img2,result),axis=1)
#cv2.namedWindow('Result',cv2.WINDOW_NORMAL)
#cv2.imshow('Result',allImg)
# cv2.imwrite('Result-all.jpg',allImg)
# cv2.imwrite('result.jpg',result)
#cv2.waitKey(0)
key_information = []
file_list = os.listdir(FILE_PATH)
for file in file_list:
    all_information = []
    file_name = FILE_PATH + '/' + file
    file_format = file_name.split('.')[-1].lower() #提取文件格式，用于判断该文件是否是图片
    if file_format == 'jpg' or file_format == 'png' or file_format == 'jpeg' or file_format == 'bmp':    #若文件是图片格式，才继续后续处理步骤
        #图像调正
        if file_name != FILE_PATH + '/' + 'template.jpg':
            preprcessed_image_name = "./temp/temp2_" + file
            sift_image = siftImageAlignment(TEMPLATE,file_name)
            #cv2.imwrite(preprcessed_image_name, sift_image)
            #preprocess(sift_image, preprcessed_image_name)
            image = get_file_content(preprcessed_image_name)  #读取图像
            result = {}
            result = client.accurate(image)            #API对图像进行识别
            if 'error_code' in result:
                result = client.basicAccurate(image)
            #print(result)
            if 'error_code' not in result:
                #print(len(result['words_result']))
                for i in range(len(result['words_result'])):
                    print((result['words_result'][i]['words']))
                    all_information.append(result['words_result'][i]['words'])  #获取识别得到的每一行的结果
            else:
                error_msg = "API调用失败:"+str(result['error_msg'])
                print(error_msg)
            #print('------')
            #print(all_information)
            key_information_dict = extract_key_information(all_information)  #根据要求提取关键信息
            key_information_dict['文件路径'] = file_name
            key_information.append(key_information_dict)
            
#创建数据库列表
DATA_BASE_NAME = 'bill-information-test.db'
creat_database(DATA_BASE_NAME)

for i in range(len(key_information)):
    try: 
        insert_into_database(key_information[i])
    except KeyError:
        print(key_information[i]['文件路径'] + '识别失败！')

   

自助回单专用凭证ICBC中国工商银行业务回单(收款)
日期:2017年11月10日
回单编号:173140000
付款人户名:安塞宏达工贸有限责任公司
付款人开户行:中国农业银行延安分行
付款人账号(卡号):26930101040004849
库管清算中心
收款人户名:涿州铁新泵业科技发展有限公司
收款人开户行:保定涿州支行营业室
收款人账号(卡号):0409020009245098508
金额:伍万元整
小写:50000.00元
业务(产品)种类:跨行收报
凭证种0000000
证00000
摘要:货款
用途:
币种:人民币
交易机构:00900200记账柜员:00023交易代码:52110渠道:其他
单机专用
附言:货款
支付交易序号
:234828报*额客户发起普通贷业日期011003
648278
业务类型(种类):普通汇兑
本回单为第1次打印,注意重复打印日期:2017年11月14日打印柜员:9验证码:E3E1FC235006
自助回单专用凭证ICBC中国工商银行业务回单(付款)
日期:2017年11月06日
回单号:1000003
付款人户名:涿州铁新泵业科技发展有限公司
付款人开户行:保定涿州支行营业室
付款人账号(卡号):0409020009245098508
收款人户名:
收款人开户行:
收款人账号(卡号):
金额:壹拾元整
小写:10.00元
业务(产品)种类:跨行发报
凭证种类000000
凭证号码0000000
摘要:跨行支付手续费
用途:
币种:人民币
交易机构:0040900200记账柜员:02226交易代码:52068渠道:其他
白单机专立
附言:年检费5522058
(003)
支付交易序号:29766290报文种类:小额客户发起普通贷记业务委托日期:2017-11-06
业务类型(种类):普通汇兑原发报金额:12,000.0
本回单为第1次打印,注意重复打印日期:201年11月14日打印柜员:9验证码:92D05D954006
自助回单专用凭证ICBC中国工商银行业务回单(付款)
日期:2017年10月12日
回单编号:172850
付款人户名:涿州铁新泵业科技发展有限公司
付款人开户行:保定涿州支行营业室
付款人账号(卡
59808
收款人户名:涿州市桃园钢化玻璃厂
收款人开户行:中国银行涿州支行
收款人账号(卡号):100148

AttributeError: 'NoneType' object has no attribute 'group'

In [1]:
import cv2
#sift = cv2.xfeatures2d_SIFT.create()
sift = cv2.xfeatures2d.SIFT_create()

In [18]:
print(all_information)

['自助回单专用凭证ICBC中国工商银行业务回单(付款)', '日期:2017年10月12日', '回单编号:172850', '付款人户名:涿州铁新泵业科技发展有限公司', '付款人开户行:保定涿州支行营业室', '付款人账号(卡', '59808', '收款人户名:涿州市桃园钢化玻璃厂', '收款人开户行:中国银行涿州支行', '收款人账号(卡号):100148062313', '金额:贰拾万元整', '小写:200,000.00', '业务(产品)种类:跨行发报', '凭证种类:转账支票', '证003882083', '摘要:退订金', '用途:', '币种人民币', '交易机构:0040900200记账柜员:01547交易代码:52068渠道:其他', '自四单机专用章', '附言:退订金', '支付交易序号:50122442报文种类:大额客户发起汇兑业务委托日期:2017-10-12', '(003)', '业务类型(种类):普通汇兑', '本回单为第1次打印,注意重复打印日期2017年11月14日打印柜员:9验证码:7 7FAEAB754006']
